In [2]:
import tiktoken

print(tiktoken.__version__)

0.12.0


In [4]:
tokenizer = tiktoken.get_encoding("gpt2")

In [6]:
uk_word = "Awirw ier"

integers = tokenizer.encode(uk_word)

In [8]:
for i in integers:
    print(f"{i} --> {tokenizer.decode(i)}")



TypeError: argument 'tokens': 'int' object cannot be converted to 'Sequence'